In [1]:
%cd drive/MyDrive/Fall23/Research/ml_1m/
!ls

/content/drive/MyDrive/Fall23/Research/datasets
amazon_beauty			    amazon_m2_dataprocessing.ipynb  ml_1m_dataprocessing.ipynb
amazon_beauty_dataprocessing.ipynb  Beauty_SASRec.txt		    ml-1m.txt
amazon_fashion			    DataProcessing.py		    Steam.txt
AmazonM2			    ml_1m			    Video.txt


In [2]:
import gzip
from collections import defaultdict
from datetime import datetime
import pandas as pd
import json

In [3]:
users_path = './ml_1m/ml-1m/users.dat'
movies_path = './ml_1m/ml-1m/movies.dat'
ratings_path = './ml_1m/ml-1m/ratings.dat'

## Creating User Profile

In [4]:
with open(users_path, 'rb') as file:
    data = file.read()

In [6]:
data = data.decode('utf-8')

# Split the data into lines
lines = data.split('\n')

# Split each line into columns (assuming it's a tab-separated file)
data_list = [line.split('::') for line in lines]

print(data_list)


[['1', 'F', '1', '10', '48067'], ['2', 'M', '56', '16', '70072'], ['3', 'M', '25', '15', '55117'], ['4', 'M', '45', '7', '02460'], ['5', 'M', '25', '20', '55455'], ['6', 'F', '50', '9', '55117'], ['7', 'M', '35', '1', '06810'], ['8', 'M', '25', '12', '11413'], ['9', 'M', '25', '17', '61614'], ['10', 'F', '35', '1', '95370'], ['11', 'F', '25', '1', '04093'], ['12', 'M', '25', '12', '32793'], ['13', 'M', '45', '1', '93304'], ['14', 'M', '35', '0', '60126'], ['15', 'M', '25', '7', '22903'], ['16', 'F', '35', '0', '20670'], ['17', 'M', '50', '1', '95350'], ['18', 'F', '18', '3', '95825'], ['19', 'M', '1', '10', '48073'], ['20', 'M', '25', '14', '55113'], ['21', 'M', '18', '16', '99353'], ['22', 'M', '18', '15', '53706'], ['23', 'M', '35', '0', '90049'], ['24', 'F', '25', '7', '10023'], ['25', 'M', '18', '4', '01609'], ['26', 'M', '25', '7', '23112'], ['27', 'M', '25', '11', '19130'], ['28', 'F', '25', '1', '14607'], ['29', 'M', '35', '7', '33407'], ['30', 'F', '35', '7', '19143'], ['31', '

In [7]:
# Convert the list of lists into a DataFrame
users_df = pd.DataFrame(data_list)
print(users_df.shape)
users_df.head()

(6041, 5)


,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
users_df_new_column_names = ['user_id', 'gender', 'age_group', 'occupation', 'zip']
users_df.columns = users_df_new_column_names
users_df.head()

,user_id,gender,age_group,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [10]:
columns_to_keep = ['user_id', 'gender', 'age_group', 'occupation']
users_df = users_df[columns_to_keep]
users_df.head()

,user_id,gender,age_group,occupation
0,1,F,1,10
1,2,M,56,16
2,3,M,25,15
3,4,M,45,7
4,5,M,25,20


In [12]:
users_df.occupation.value_counts()

4     759
0     711
7     679
1     528
17    502
12    388
14    302
20    281
2     267
16    241
6     236
10    195
3     173
15    144
13    142
11    129
5     112
9      92
19     72
18     70
8      17
Name: occupation, dtype: int64

In [17]:
users_df.iloc[0]['occupation']

'10'

In [22]:
occupation_dict = {
    0:  "unknown", 1:  "academic/educator", 2:  "artist", 3:  "clerical/admin",
    4:  "college/grad student", 5:  "customer service", 6:  "doctor/health care",
    7:  "executive/managerial", 8:  "farmer", 9:  "homemaker", 10:  "k-12 student",
    11:  "lawyer", 12:  "programmer", 13:  "retired", 14:  "sales/marketing",
    15:  "scientist", 16:  "self-employed", 17:  "technician/engineer",
    18:  "tradesman/craftsman", 19:  "unemployed", 20:  "writer"
}
print(occupation_dict)

{0: 'unknown', 1: 'academic/educator', 2: 'artist', 3: 'clerical/admin', 4: 'college/grad student', 5: 'customer service', 6: 'doctor/health care', 7: 'executive/managerial', 8: 'farmer', 9: 'homemaker', 10: 'k-12 student', 11: 'lawyer', 12: 'programmer', 13: 'retired', 14: 'sales/marketing', 15: 'scientist', 16: 'self-employed', 17: 'technician/engineer', 18: 'tradesman/craftsman', 19: 'unemployed', 20: 'writer'}


In [23]:
def getOccuptaion(x):
  try:
    return occupation_dict[int(x)]
  except:
    pass
  return x

users_df['new_occupation'] = users_df.apply(lambda x: getOccuptaion(x['occupation']), axis = 1)
users_df.head()

,user_id,gender,age_group,occupation,new_occupation
0,1,F,1,10,k-12 student
1,2,M,56,16,self-employed
2,3,M,25,15,scientist
3,4,M,45,7,executive/managerial
4,5,M,25,20,writer


In [24]:
users_df.new_occupation.value_counts()

college/grad student    759
unknown                 711
executive/managerial    679
academic/educator       528
technician/engineer     502
programmer              388
sales/marketing         302
writer                  281
artist                  267
self-employed           241
doctor/health care      236
k-12 student            195
clerical/admin          173
scientist               144
retired                 142
lawyer                  129
customer service        112
homemaker                92
unemployed               72
tradesman/craftsman      70
farmer                   17
Name: new_occupation, dtype: int64

In [25]:
age_dict = {
    1:  "under 18", 18: "between 18 to 24", 25:  "between 25 to 34",
    35:  "between 35 to 44", 45:  "between 45 to 49",
    50:  "between 50 to 55", 56:  "above 56"
}
print(age_dict)

{1: 'under 18', 18: 'between 18 to 24', 25: 'between 25 to 34', 35: 'between 35 to 44', 45: 'between 45 to 49', 50: 'between 50 to 55', 56: 'above 56'}


In [26]:
def setAge(x):
  try:
    return age_dict[int(x)]
  except:
    pass
  return x

users_df['new_age'] = users_df.apply(lambda x: setAge(x['age_group']), axis = 1)
users_df.head()

,user_id,gender,age_group,occupation,new_occupation,new_age
0,1,F,1,10,k-12 student,under 18
1,2,M,56,16,self-employed,above 56
2,3,M,25,15,scientist,between 25 to 34
3,4,M,45,7,executive/managerial,between 45 to 49
4,5,M,25,20,writer,between 25 to 34


In [28]:
def getUserProfile(gender, new_age, new_occupation):
  try:
    if gender == 'F':
      gender = 'female'
    else:
      gender = 'male'
    user_profile = f'{gender} {new_occupation} {new_age} years of age'
    return user_profile
  except:
    pass
  return f'{gender} {new_occupation} {new_age} years of age'

users_df['user_profile'] = users_df.apply(lambda x: getUserProfile(x['gender'], x['new_age'], x['new_occupation']), axis = 1)
users_df.head()

,user_id,gender,age_group,occupation,new_occupation,new_age,user_profile
0,1,F,1,10,k-12 student,under 18,female k-12 student under 18 years of age
1,2,M,56,16,self-employed,above 56,male self-employed above 56 years of age
2,3,M,25,15,scientist,between 25 to 34,male scientist between 25 to 34 years of age
3,4,M,45,7,executive/managerial,between 45 to 49,male executive/managerial between 45 to 49 yea...
4,5,M,25,20,writer,between 25 to 34,male writer between 25 to 34 years of age


In [29]:
columns_to_keep = ['user_id', 'user_profile']
users_df = users_df[columns_to_keep]
users_df.head()

,user_id,user_profile
0,1,female k-12 student under 18 years of age
1,2,male self-employed above 56 years of age
2,3,male scientist between 25 to 34 years of age
3,4,male executive/managerial between 45 to 49 yea...
4,5,male writer between 25 to 34 years of age


In [30]:
users_df.to_csv('./ml_1m/ml-1m/user_profile.csv')

## Movies Data cleaning

In [34]:
with open(movies_path, 'rb') as file:
    data = file.read()
data = data.decode(encoding = "ISO-8859-1")

# Split the data into lines
lines = data.split('\n')

# Split each line into columns (assuming it's a tab-separated file)
data_list = [line.split('::') for line in lines]

print(data_list)

[['1', 'Toy Story (1995)', "Animation|Children's|Comedy"], ['2', 'Jumanji (1995)', "Adventure|Children's|Fantasy"], ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'], ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama'], ['5', 'Father of the Bride Part II (1995)', 'Comedy'], ['6', 'Heat (1995)', 'Action|Crime|Thriller'], ['7', 'Sabrina (1995)', 'Comedy|Romance'], ['8', 'Tom and Huck (1995)', "Adventure|Children's"], ['9', 'Sudden Death (1995)', 'Action'], ['10', 'GoldenEye (1995)', 'Action|Adventure|Thriller'], ['11', 'American President, The (1995)', 'Comedy|Drama|Romance'], ['12', 'Dracula: Dead and Loving It (1995)', 'Comedy|Horror'], ['13', 'Balto (1995)', "Animation|Children's"], ['14', 'Nixon (1995)', 'Drama'], ['15', 'Cutthroat Island (1995)', 'Action|Adventure|Romance'], ['16', 'Casino (1995)', 'Drama|Thriller'], ['17', 'Sense and Sensibility (1995)', 'Drama|Romance'], ['18', 'Four Rooms (1995)', 'Thriller'], ['19', 'Ace Ventura: When Nature Calls (1995)', 'Comedy'], ['20', 'Mone

In [35]:
# Convert the list of lists into a DataFrame
movies_df = pd.DataFrame(data_list)
print(movies_df.shape)
movies_df.head()

(3884, 3)


,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [36]:
movies_df_new_columns = ['movie_id', 'movie_name', 'genre']
movies_df.columns = movies_df_new_columns
movies_df.head()

,movie_id,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [50]:
def getMovieName(name):
  try:
    return name.strip()[:-6]
  except:
    pass
  return name

def getMovieYear(name):
  try:
    return name.strip()[-5:-1]
  except:
    pass
  return name

movies_df['year'] = movies_df.apply(lambda x: getMovieYear(x['movie_name']), axis = 1)
movies_df['name'] = movies_df.apply(lambda x: getMovieName(x['movie_name']), axis = 1)
movies_df.head()

,movie_id,movie_name,genre,year,name
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama,1995,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II


In [51]:
def cleanGenre(genre):
  try:
    return ','.join(genre.strip().split('|'))
  except:
    pass
  return genre

movies_df['cleaned_genre'] = movies_df.apply(lambda x: cleanGenre(x['genre']), axis = 1)
movies_df.head()

,movie_id,movie_name,genre,year,name,cleaned_genre
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,"Animation,Children's,Comedy"
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995,Jumanji,"Adventure,Children's,Fantasy"
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,Grumpier Old Men,"Comedy,Romance"
3,4,Waiting to Exhale (1995),Comedy|Drama,1995,Waiting to Exhale,"Comedy,Drama"
4,5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II,Comedy


In [52]:
columns_to_keep = ['movie_id', 'name', 'cleaned_genre', 'year']
movies_df = movies_df[columns_to_keep]
movies_df.head()

,movie_id,name,cleaned_genre,year
0,1,Toy Story,"Animation,Children's,Comedy",1995
1,2,Jumanji,"Adventure,Children's,Fantasy",1995
2,3,Grumpier Old Men,"Comedy,Romance",1995
3,4,Waiting to Exhale,"Comedy,Drama",1995
4,5,Father of the Bride Part II,Comedy,1995


In [53]:
movies_df.to_csv('./ml_1m/ml-1m/movies.csv')

In [4]:
with open(ratings_path, 'rb') as file:
    data = file.read()
data = data.decode(encoding = "ISO-8859-1")

# Split the data into lines
lines = data.split('\n')

# Split each line into columns (assuming it's a tab-separated file)
data_list = [line.split('::') for line in lines]

print(data_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
# Convert the list of lists into a DataFrame
ratings_df = pd.DataFrame(data_list)
print(ratings_df.shape)
ratings_df.head()

(1000210, 4)


,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
ratings_df_new_columns = ['user_id', 'movie_id', 'rating', 'time']
ratings_df.columns = ratings_df_new_columns
ratings_df.head()

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
final_ratings_df = ratings_df.groupby('user_id').apply(lambda x: x.sort_values(by='time')).reset_index(drop=True)
final_ratings_df.head(50)

,user_id,movie_id,rating,time
0,,None,None,None
1,1,3186,4,978300019
2,1,1721,4,978300055
3,1,1022,5,978300055
4,1,1270,5,978300055
5,1,2340,3,978300103
6,1,1836,5,978300172
7,1,3408,4,978300275
8,1,1207,4,978300719
9,1,2804,5,978300719


In [8]:
final_ratings_df.user_id.value_counts()

4169    2314
1680    1850
4277    1743
1941    1595
1181    1521
        ... 
3291      20
2160      20
217       20
4419      20
           1
Name: user_id, Length: 6041, dtype: int64

In [9]:
final_ratings_df.to_csv('./ml_1m/ml-1m/ratings.csv')